In [1]:
import numpy as np
import pandas as pd

In [2]:
housing = pd.read_csv("/Users/naoki/train.csv")

In [3]:
housing.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [4]:
X = housing[["GrLivArea", "YearBuilt"]]
X.head()

,GrLivArea,YearBuilt
0,1710,2003
1,1262,1976
2,1786,2001
3,1717,1915
4,2198,2000


In [5]:
y = housing["SalePrice"]
y.head()

0    208500
1    181500
2    223500
3    140000
4    250000
Name: SalePrice, dtype: int64

In [6]:
from sklearn.model_selection import train_test_split
(X_train, X_test,
 y_train, y_test) = train_test_split(
    X, y, test_size=0.3, random_state=0, shuffle=True
)

In [7]:
X_train.head()

,GrLivArea,YearBuilt
64,2034,1997
682,1291,1996
960,858,1958
1384,1258,1939
1100,438,1920


In [8]:
X_train.head()

,GrLivArea,YearBuilt
64,2034,1997
682,1291,1996
960,858,1958
1384,1258,1939
1100,438,1920


 ## 【問題1】ブレンディングのスクラッチ実装
ブレンディング をスクラッチ実装し、単一モデルより精度があがる例を 最低3つ 示してください。精度があがるとは、検証用データに対する平均二乗誤差（MSE）が小さくなることを指します。

In [9]:
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor

In [10]:
from sklearn import linear_model
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression

In [11]:
LM= linear_model.LinearRegression(normalize=True)
print(LM)
LM.fit(X_train, y_train)
LM_pred = LM.predict(X_test)
print("MSE(linear_model)",mean_squared_error(y_test, LM_pred))


sc = StandardScaler()
X_test_std = sc.fit_transform(X_test)
X_train_std = sc.fit_transform(X_train)
SV= SVR(kernel='rbf')
SV.fit(X_train_std, y_train)
SV_pred = SV.predict(X_test_std)
print("MSE(SVR)",mean_squared_error(y_test, SV_pred))


DR = DecisionTreeRegressor(max_depth=3)
DR.fit(X_train, y_train)
DR_pred = DR.predict(X_test)
print("MSE(DecisionTreeRegressor)",mean_squared_error(y_test, DR_pred))

mix_pred = np.vstack((LM_pred, SV_pred, DR_pred))
mix_pred.shape

mean_pred = LM_pred*0.5+SV_pred*0.1+DR_pred*0.4
print("MSE(mean)",mean_squared_error(y_test, mean_pred))

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=True)
MSE(linear_model) 2690647926.377604
MSE(SVR) 7119069689.683064
MSE(DecisionTreeRegressor) 2553443455.30812
MSE(mean) 2470929971.5410123


In [12]:
from sklearn.linear_model import Lasso

Las= Lasso(alpha=0.1)
# print(Las)
Las.fit(X_train, y_train)
Las_pred = Las.predict(X_test)
print("MSE(Lasso)",mean_squared_error(y_test, Las_pred))


sc = StandardScaler()
X_test_std = sc.fit_transform(X_test)
X_train_std = sc.fit_transform(X_train)
SV= SVR(kernel='rbf',C=0.5)
# print(SV)
SV.fit(X_train_std, y_train)
SV_pred = SV.predict(X_test_std)
print("MSE(SVR)",mean_squared_error(y_test, SV_pred))


DR = DecisionTreeRegressor(max_depth=3)
DR.fit(X_train, y_train)
DR_pred = DR.predict(X_test)
print("MSE(DecisionTreeRegressor)",mean_squared_error(y_test, DR_pred))

mix_pred = np.vstack((LM_pred, SV_pred, DR_pred))
mix_pred.shape

mean_pred = Las_pred*0.5+SV_pred*0.05+DR_pred*0.45
print("MSE(mean)",mean_squared_error(y_test, mean_pred))

MSE(Lasso) 2690647942.4335794
MSE(SVR) 7128920068.0851145
MSE(DecisionTreeRegressor) 2553443455.30812
MSE(mean) 2413207951.9277983


In [13]:
from sklearn.linear_model import Ridge

Rid= Ridge(alpha=0.1)
# print(Las)
Rid.fit(X_train, y_train)
Rid_pred = Rid.predict(X_test)
print("MSE(Ridge)",mean_squared_error(y_test, Rid_pred))


sc = StandardScaler()
X_test_std = sc.fit_transform(X_test)
X_train_std = sc.fit_transform(X_train)
SV= SVR(kernel='linear', C=100)
# print(SV)
SV.fit(X_train_std, y_train)
SV_pred = SV.predict(X_test_std)
print("MSE(SVR)",mean_squared_error(y_test, SV_pred))


DR = DecisionTreeRegressor(max_depth=9)
DR.fit(X_train, y_train)
DR_pred = DR.predict(X_test)
print("MSE(DecisionTreeRegressor)",mean_squared_error(y_test, DR_pred))

mix_pred = np.vstack((LM_pred, SV_pred, DR_pred))
mix_pred.shape

mean_pred = Rid_pred*0.2+SV_pred*0.1+DR_pred*0.7
print("MSE(mean)",mean_squared_error(y_test, mean_pred))

MSE(Ridge) 2690647942.0877905
MSE(SVR) 3389628361.3601527
MSE(DecisionTreeRegressor) 1799261318.0862226
MSE(mean) 1753122620.1441343


 ## 【問題2】バギングのスクラッチ実装
バギング をスクラッチ実装し、単一モデルより精度があがる例を 最低1つ 示してください。

In [14]:
from sklearn.ensemble import GradientBoostingRegressor
import lightgbm as lgb
import xgboost as xgb

In [15]:
np.random.seed(1)
rand_index = np.random.choice(X_train.index,1022, replace=True)
np.unique(rand_index).shape


(649,)

In [16]:
np.random.seed(1)
rand_index = np.random.choice(X_train.index,X_train.shape[0], replace=True)
X_train1 = X_train.loc[rand_index]
y_train1 = y_train.loc[rand_index]

np.random.seed(2)
rand_index = np.random.choice(X_train.index,X_train.shape[0], replace=True)
X_train2 = X_train.loc[rand_index]
y_train2 = y_train.loc[rand_index]

np.random.seed(3)
rand_index = np.random.choice(X_train.index,X_train.shape[0], replace=True).tolist()
X_train3 = X_train.loc[rand_index]
y_train3 = y_train.loc[rand_index]

In [17]:
reg1 = GradientBoostingRegressor(n_estimators=100,max_depth=5).fit(X_train1, y_train1)
y_pred1 =reg1.predict(X_test)
print("MSE(reg1)",mean_squared_error(y_test, y_pred1))

reg2 = GradientBoostingRegressor(n_estimators=500,max_depth=3).fit(X_train2, y_train2)
y_pred2 =reg2.predict(X_test)
print("MSE(reg2)",mean_squared_error(y_test, y_pred2))

reg3 = GradientBoostingRegressor(n_estimators=250,max_depth=5).fit(X_train3, y_train3)
y_pred3 =reg3.predict(X_test)
print("MSE(reg2)",mean_squared_error(y_test, y_pred3))
print("==================")
y_pred_mean = (y_pred1 + y_pred2 + y_pred3)/3
print("MSE(mean)",mean_squared_error(y_test, y_pred_mean))

MSE(reg1) 1955856381.8803396
MSE(reg2) 2246120995.481195
MSE(reg2) 1725817235.2522635
MSE(mean) 1659935727.6287725


In [18]:
reg1 = xgb.XGBRegressor(n_estimators=100,max_depth=5).fit(X_train1, y_train1)
y_pred1 =reg1.predict(X_test)
print("MSE(reg1)",mean_squared_error(y_test, y_pred1))

reg2 = xgb.XGBRegressor(n_estimators=500,max_depth=3).fit(X_train2, y_train2)
y_pred2 =reg2.predict(X_test)
print("MSE(reg2)",mean_squared_error(y_test, y_pred2))

reg3 = xgb.XGBRegressor(n_estimators=250,max_depth=5).fit(X_train3, y_train3)
y_pred3 =reg3.predict(X_test)
print("MSE(reg2)",mean_squared_error(y_test, y_pred3))

print("==================")
y_pred_mean = (y_pred1 + y_pred2 + y_pred3)/3
print("MSE(mean)",mean_squared_error(y_test, y_pred_mean))

/Users/naoki/opt/anaconda3/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/Users/naoki/opt/anaconda3/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/Users/naoki/opt/anaconda3/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[18:50:25] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
MSE(reg1) 2213829666.0405197
[18:50:26] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
MSE(reg2) 2247233980.532478
[18:50:26] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
MSE(reg2) 1705477084.5991666
MSE(mean) 1687972552.6029377


In [19]:
reg1 = lgb.LGBMRegressor(n_estimators=100,max_depth=5).fit(X_train1, y_train1)
y_pred1 =reg1.predict(X_test)
print("MSE(reg1)",mean_squared_error(y_test, y_pred1))

reg2 = lgb.LGBMRegressor(n_estimators=500,max_depth=3).fit(X_train2, y_train2)
y_pred2 =reg2.predict(X_test)
print("MSE(reg2)",mean_squared_error(y_test, y_pred2))

reg3 = lgb.LGBMRegressor(n_estimators=250,max_depth=5).fit(X_train3, y_train3)
y_pred3 =reg3.predict(X_test)
print("MSE(reg2)",mean_squared_error(y_test, y_pred3))
print("==================")
y_pred_mean = (y_pred1 + y_pred2 + y_pred3)/3
print("MSE(mean)",mean_squared_error(y_test, y_pred_mean))
y_pred3[1]

MSE(reg1) 1996989149.286145
MSE(reg2) 2217785612.142733
MSE(reg2) 2337926346.668943
MSE(mean) 2010251814.6958816


149261.18481853406

In [20]:
class Bagging():

#     def __init__(self, n_clusters, max_iter, verbose=False, tol=5, seed=0):
#         # ハイパーパラメータを属性として記録
#         self.n_clusters = n_clusters
#         self.seed = seed
#         self.n_init = n_init
#         self.max_iter = max_iter
#         self.tol= tol
#         self.verbose = verbose
        
    def fit(self, models, X, y):
        self.model_list = []
        for j, model in enumerate(models):
            np.random.seed(j)
            self.rand_index = np.random.choice(X.index,X.shape[0], replace=True)
            self.X_rand = X.loc[self.rand_index]
            self.y_rand = y.loc[self.rand_index]
            self.model_list.append(model.fit(self.X_rand, self.y_rand))
            
    def predict(self, X, y):
        print(X.shape[0])
        print(len(self.model_list))
        self.pred_data = np.zeros((X.shape[0],len(self.model_list)))
        for i, model in enumerate(self.model_list):
            self.pred = model.predict(X)
            self.pred_data[:,i] = self.pred
        self.final_pred = np.mean(self.pred_data, axis=1)
        print("MSE(mean)",mean_squared_error(y, self.final_pred))
        return self.final_pred
        

In [21]:
models = [xgb.XGBRegressor(n_estimators=100,max_depth=5), GradientBoostingRegressor(n_estimators=100,max_depth=5), lgb.LGBMRegressor(n_estimators=100,max_depth=5)]
bg = Bagging()
bg.fit(models=models, X = X_train, y = y_train)

[18:50:34] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


/Users/naoki/opt/anaconda3/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


In [22]:
bg.predict(X_test, y_test)[:10]

438
3
MSE(mean) 1836970587.4318545


array([206735.96384136, 152325.14244614, 128424.80811652, 171632.29484512,
       125163.23555442,  95525.13495285, 200498.04594153, 123503.19352971,
       562100.2163109 , 147588.21288913])

In [23]:
lgb.LGBMRegressor

lightgbm.sklearn.LGBMRegressor

In [24]:
models = [lgb.LGBMRegressor(n_estimators=100,max_depth=5), lgb.LGBMRegressor(n_estimators=100,max_depth=5), lgb.LGBMRegressor(n_estimators=100,max_depth=5)]
bg = Bagging()
bg.fit(models=models, X = X_train, y = y_train)

In [25]:
bg.predict(X_test, y_test)[:10]

438
3
MSE(mean) 1905905323.963103


array([186616.01381421, 154654.02881744, 128076.41901312, 184057.70727728,
       128618.50371924,  91991.24366953, 202164.25872405, 126932.11921891,
       441678.27135167, 146946.96392326])

In [26]:
models = [xgb.XGBRegressor(n_estimators=100,max_depth=5), xgb.XGBRegressor(n_estimators=100,max_depth=5), xgb.XGBRegressor(n_estimators=100,max_depth=5)]
bg = Bagging()
bg.fit(models=models, X = X_train, y = y_train)

[18:50:38] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:50:38] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:50:38] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


/Users/naoki/opt/anaconda3/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


In [27]:
bg.predict(X_test, y_test)[:10]

438
3
MSE(mean) 1792487618.8477426


array([218511.16145833, 154213.35416667, 128110.38020833, 177497.30729167,
       128028.48958333,  94287.4296875 , 205607.26041667, 123691.86458333,
       574186.22916667, 149404.109375  ])

In [28]:
models = [xgb.XGBRegressor(n_estimators=100,max_depth=5), lgb.LGBMRegressor(n_estimators=100,max_depth=5), GradientBoostingRegressor(n_estimators=100,max_depth=5)]
bg = Bagging()
bg.fit(models=models, X = X_train, y = y_train)

[18:50:41] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


/Users/naoki/opt/anaconda3/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


In [29]:
bg.predict(X_test, y_test)[:10]

438
3
MSE(mean) 1818920351.7739475


array([216351.44617371, 155349.8072607 , 127352.45661918, 173557.16653544,
       127557.65941865,  94811.82851673, 206640.40675438, 120714.07235049,
       478249.05276656, 151819.7788324 ])

 ## 【問題3】スタッキングのスクラッチ実装
スタッキング をスクラッチ実装し、単一モデルより精度があがる例を 最低1つ 示してください。

In [30]:
X1_train, X2_train, X3_train  = np.split(X_train, [340, 681])
y1_train, y2_train, y3_train  = np.split(y_train, [340, 681])

In [31]:
print("X1_train",X1_train.shape)
print("X2_train",X2_train.shape)
print("X3_train",X3_train.shape)

X1_train (340, 2)
X2_train (341, 2)
X3_train (341, 2)


In [32]:
print("y1_train",y1_train.shape)
print("y2_train",y2_train.shape)
print("y3_train",y3_train.shape)

y1_train (340,)
y2_train (341,)
y3_train (341,)


In [33]:
X_train1 = np.vstack((X2_train, X3_train))
y_train1 = np.hstack((y2_train, y3_train))

reg1_Gr = GradientBoostingRegressor(n_estimators=100,max_depth=5).fit(X_train1, y_train1)
y_pred1_reg1_Gr =reg1_Gr.predict(X1_train)
print("MSE(reg1)",mean_squared_error(y1_train, y_pred1_reg1_Gr))


X_train2 = np.vstack((X1_train, X3_train))
y_train2 = np.hstack((y1_train, y3_train))

reg2_Gr = GradientBoostingRegressor(n_estimators=500,max_depth=3).fit(X_train2, y_train2)
y_pred2_reg2_Gr =reg2_Gr.predict(X2_train)
print("MSE(reg2)",mean_squared_error(y2_train, y_pred2_reg2_Gr))


X_train3 = np.vstack((X1_train, X2_train))
y_train3 = np.hstack((y1_train, y2_train))


reg3_Gr = GradientBoostingRegressor(n_estimators=250,max_depth=5).fit(X_train3, y_train3)
y_pred3_reg3_Gr =reg3_Gr.predict(X3_train)
print("MSE(reg2)",mean_squared_error(y3_train, y_pred3_reg3_Gr))

y_train_Gr = np.hstack((y_pred1_reg1_Gr, y_pred2_reg2_Gr, y_pred2_reg2_Gr))

MSE(reg1) 2071666887.9999838
MSE(reg2) 2375194881.2290134
MSE(reg2) 1451548775.9864154


In [34]:
X_train1 = np.vstack((X2_train, X3_train))
y_train1 = np.hstack((y2_train, y3_train))

reg1_LG = lgb.LGBMRegressor(n_estimators=100,max_depth=5).fit(X_train1, y_train1)
y_pred1_reg1_LG =reg1_LG.predict(X1_train)
print("MSE(reg1)",mean_squared_error(y1_train, y_pred1_reg1_LG))


X_train2 = np.vstack((X1_train, X3_train))
y_train2 = np.hstack((y1_train, y3_train))

reg2_LG = lgb.LGBMRegressor(n_estimators=500,max_depth=3).fit(X_train2, y_train2)
y_pred2_reg2_LG =reg2_LG.predict(X2_train)
print("MSE(reg2)",mean_squared_error(y2_train, y_pred2_reg2_LG))


X_train3 = np.vstack((X1_train, X2_train))
y_train3 = np.hstack((y1_train, y2_train))


reg3_LG = lgb.LGBMRegressor(n_estimators=250,max_depth=5).fit(X_train3, y_train3)
y_pred3_reg3_LG =reg3_LG.predict(X3_train)
print("MSE(reg2)",mean_squared_error(y3_train, y_pred3_reg3_LG))

y_train_LG = np.hstack((y_pred1_reg1_LG, y_pred2_reg2_LG, y_pred2_reg2_LG))

MSE(reg1) 1789291655.0779407
MSE(reg2) 2088962682.1700788
MSE(reg2) 1383718320.4924493


In [35]:
X_train1 = np.vstack((X2_train, X3_train))
y_train1 = np.hstack((y2_train, y3_train))

reg1_RF = RandomForestRegressor(n_estimators=100,max_depth=5).fit(X_train1, y_train1)
y_pred1_reg1_RF =reg1_RF.predict(X1_train)
print("MSE(reg1)",mean_squared_error(y1_train, y_pred1_reg1_RF))


X_train2 = np.vstack((X1_train, X3_train))
y_train2 = np.hstack((y1_train, y3_train))

reg2_RF = RandomForestRegressor(n_estimators=500,max_depth=3).fit(X_train2, y_train2)
y_pred2_reg2_RF =reg2_RF.predict(X2_train)
print("MSE(reg2)",mean_squared_error(y2_train, y_pred2_reg2_RF))


X_train3 = np.vstack((X1_train, X2_train))
y_train3 = np.hstack((y1_train, y2_train))


reg3_RF = RandomForestRegressor(n_estimators=250,max_depth=5).fit(X_train3, y_train3)
y_pred3_reg3_RF =reg3_RF.predict(X3_train)
print("MSE(reg2)",mean_squared_error(y3_train, y_pred3_reg3_RF))

y_train_RF = np.hstack((y_pred1_reg1_RF, y_pred2_reg2_RF, y_pred2_reg2_RF))

MSE(reg1) 2000503394.4372175
MSE(reg2) 2426725876.5467005
MSE(reg2) 1180293786.2972999


In [36]:
train_mix = np.vstack((y_train_Gr, y_train_LG, y_train_RF)).T
train_mix.shape

(1022, 3)

In [37]:
xgb.XGBRegressor
reg_end = xgb.XGBRegressor(n_estimators=250,max_depth=5).fit(train_mix, y_train)


[18:50:51] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


/Users/naoki/opt/anaconda3/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


In [38]:
X_test1_Gr =reg1_Gr.predict(X_test)
X_test2_Gr =reg2_Gr.predict(X_test)
X_test3_Gr = reg3_Gr.predict(X_test)

y_test_Gr = (X_test1_Gr + X_test2_Gr + X_test3_Gr) /3

In [39]:
y_test_Gr.shape

(438,)

In [40]:
X_test1_LG =reg1_LG.predict(X_test)
X_test2_LG =reg2_LG.predict(X_test)
X_test3_LG = reg3_LG.predict(X_test)

y_test_LG = (X_test1_LG + X_test2_LG + X_test3_LG) /3


In [41]:
y_test_LG.shape

(438,)

In [42]:
X_test1_RF =reg1_RF.predict(X_test)
X_test2_RF =reg2_RF.predict(X_test)
X_test3_RF = reg3_RF.predict(X_test)

y_test_RF = (X_test1_RF + X_test2_RF + X_test3_RF) /3

In [43]:
y_test_RF.shape

(438,)

In [44]:
X_test_end = np.vstack((y_test_Gr,y_test_LG,y_test_RF)).T
X_test_end.shape

(438, 3)

In [45]:
y_pred_end = reg_end.predict(X_test_end)
y_pred_end.shape

(438,)

In [46]:
print("MSE(Stacking)",mean_squared_error(y_test, y_pred_end))


MSE(Stacking) 3447336826.849463


In [47]:
from sklearn.model_selection import KFold

In [48]:
# class Stacking():
#     def __init__(self,models):
#         self.models = models

# #     def __init__(self, n_clusters, max_iter, verbose=False, tol=5, seed=0):
# #         # ハイパーパラメータを属性として記録
# #         self.n_clusters = n_clusters
# #         self.seed = seed
# #         self.n_init = n_init
# #         self.max_iter = max_iter
# #         self.tol= tol
# #         self.verbose = verbose

# #     def fit(self,X,y,k):
# #         self.X_y = pd.concat((X,y), axis=1)
# #         Split = 0
# #         self.Sp_list = []
# #         for i in range(k-1):
# #             Split += (X_y.shape[0] / k)
# #             self.Sp_list.append(Split)

#     def fit(self,X,y,K,seed):
#         KF = KFold(n_splits=K, random_state=seed, shuffle=True)
#         self.pred_data = np.array([])
# #         self.prad_data = pd.DataFrame()
#         X = np.array(X)
#         y = np.array(y)
#         self.preds = np.array([])
#         for i, model in enumerate(self.models):
#             for train_index, test_index in KF.split(X):
# #                 print(train_index.shape)
# #                 print(X.shape)
# #                 print(test_index.shape)
                
#                 model.fit(X[train_index],y[train_index])
#                 y_pred = model.predict(X[test_index])
# #                 print(len(y_pred))
#                 self.preds = np.append(self.preds, y_pred)
#             print(self.preds.shape)
# #             self.pred_data = np.append(self.pred_data,self.preds)
# #         print(self.pred_data.shape,"/////")
#         self.pres.reshape(len(self.models, len(X))
# #         self.pred_data.reshape(len(self.models), len(X))            

In [49]:
class Stacking():
    def __init__(self,models,end_model):
        self.models = models
        self.end_model = end_model


    def fit(self,X,y,K,seed):
        self.K = K
        KF = KFold(n_splits=K, random_state=seed, shuffle=True)
        self.pred_data = np.array([])
#         self.prad_data = pd.DataFrame()
        X = np.array(X)
        y = np.array(y)
        self.preds = np.array([])
        self.model_list = []
        for i, model in enumerate(self.models):
            for train_index, test_index in KF.split(X):
                
                self.model_list.append(model.fit(X[train_index],y[train_index]))
                y_pred = model.predict(X[test_index])

                self.preds = np.append(self.preds, y_pred)
#             print(self.preds.shape)
        self.preds = self.preds.reshape(len(self.models),X.shape[0]).T
#         print(self.preds.shape,"//////")
        self.end_model.fit(self.preds, y)
#         print(self.end_model)
        
        
    def predict(self,X, y):
        X = np.array(X)
        y = np.array(y)
        self.test_array = np.array([])
        self.final_test = np.zeros((X.shape[0],self.K))
        for model in self.model_list:
            y_pred_test = model.predict(X)
            self.test_array = np.append(self.test_array, y_pred_test)
#             print(self.test_array.shape,"test_array")
        self.test_array=self.test_array.reshape(len(self.models)*self.K, X.shape[0]).T
        print(self.test_array.shape,"test_array")
        for j, i in enumerate(range(0,len(self.models)*self.K-self.K, self.K)):
            self.mean_pred = np.mean(self.test_array[:,i:i+self.K], axis=1)
            self.final_test[:,j] = self.mean_pred
        self.final_pred = self.end_model.predict(self.final_test)
        print(self.final_test.shape,"final_test")
        return self.final_pred


In [50]:
models = [xgb.XGBRegressor(n_estimators=100,max_depth=5), GradientBoostingRegressor(n_estimators=100,max_depth=5), lgb.LGBMRegressor(n_estimators=100,max_depth=5)]

In [51]:
St = Stacking(models,end_model=GradientBoostingRegressor(n_estimators=100,max_depth=5))
St.fit(X=X_train, y=y_train,K=3,seed=0)
St.pred_data.shape



[18:51:03] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:51:03] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:51:03] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(0,)

In [52]:
y_pred_end = St.predict(X_test, y_test)
print("MSE(Stacking)",mean_squared_error(y_test, y_pred_end))

(438, 9) test_array
(438, 3) final_test
MSE(Stacking) 20217299221.598576
